In [1]:
import glob

import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

In [2]:
def append_files_from_path(path: str) -> str:
    """Given a path to a list of .txt files return a string
    of the concatenated contents of the file."""
    paths = glob.glob(path + "/*.txt")
    text = ''
    for path in paths:
        with open(path, 'r') as f:
            read_text = f.read()
            text += read_text
    return text

In [3]:
text = append_files_from_path('../data/seuss/text')

In [6]:
import spacy
from spacy.vocab import Vocab

parser = spacy.load('en', disable=['ner', 'textcat'], max_length=4000000)
parsed_text = parser(text)
tokens = tuple(set([token.text for token in parsed_text]))

int2token = dict(enumerate(tokens))
token2int = {tok: i for i, tok in int2token.items()}

In [7]:
# Encode the text
encoded = np.array([token2int[tok.text] for tok in parsed_text])

In [8]:
# # We create two dictionaries:
# # 1. int2char, which maps integers to characters
# # 2. char2int, which maps characters to integers
# chars = tuple(set(text))
# int2char = dict(enumerate(chars))
# char2int = {ch: ii for ii, ch in int2char.items()}

# # Encode the text
# encoded = np.array([char2int[ch] for ch in text])
# encoded

In [9]:
# Defining method to encode one hot labels
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [10]:
# Defining method to make mini-batches for training
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [11]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [12]:
# Declaring the model
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.tokens = tokens
        self.int2token = dict(enumerate(self.tokens))
        self.token2int = {tok: i for i, tok in self.int2token.items()}
        
        #define the LSTM
        self.lstm = nn.LSTM(len(self.tokens), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.tokens))
        
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [13]:
# Declaring the train method
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_tokens = len(net.tokens)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_tokens)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_tokens)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))


In [14]:
n_hidden=512
n_layers=2
drop_prob = 0.30

net = CharRNN(tokens, n_hidden, n_layers, drop_prob)
print(net)

# Declaring the hyperparameters
batch_size = 512
seq_length = 24
n_epochs = 5 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.01, print_every=10)

CharRNN(
  (lstm): LSTM(3003, 512, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=3003, bias=True)
)


In [15]:
# Defining a method to generate the next token
def predict(net, token, h=None, top_k=None):
        ''' Given a token, predict the next token.
            Returns the predicted token and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.token2int[token]]])
        x = one_hot_encode(x, len(net.tokens))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the token probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top tokens
        if top_k is None:
            top_tok = np.arange(len(net.tokens))
        else:
            p, top_tok = p.topk(top_k)
            top_tok = top_tok.numpy().squeeze()
        
        # select the likely next token with some element of randomness
        p = p.numpy().squeeze()
        tok = np.random.choice(top_tok, p=p/p.sum())
        
        # return the encoded value of the predicted tok and the hidden state
        return net.int2token[tok], h
        
# Declaring a method to generate new text
def sample(net, size, prime=['The', 'cat'], top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime tokens
    tokens = [tok for tok in prime]
    h = net.init_hidden(1)
    for tok in prime:
        token, h = predict(net, tok, h, top_k=top_k)

    tokens.append(token)
    
    # Now pass in the previous token and get a new one
    for i in range(size):
        token, h = predict(net, tokens[-1], h, top_k=top_k)
        tokens.append(token)

    return ''.join(tokens)

In [17]:
print(sample(net, 20, prime=['The'], top_k=5))

The


I
I
.!.!the
.
I!
the


